| [**Overview**](./00_overview.ipynb) | [**EDA**](./01_EDA.ipynb) | [**Using `sklearn` Models**](./02_LoadModels.ipynb) | [**Making Predictions**](./03_Predictions.ipynb)|
| -- | -- | -- | -- |



# Exploratory Geochemical Data Analyis in Python

This notebook is intended as a lightning introduction to what you can do in Python. In this notebook we'll:
* Introduce Python and Jupyter
* Introduce some of the key packages we'll be using: `matplotlib`, `Pandas` and `pyrolite`
* Load up some data, do some basic analysis, and make some simple plots

---
### What is Jupyter?

[Jupyter](https://jupyter.org/) is an ecosystem of open source tools which provide interfaces for working with a variety of programming languages. The most well known of these is the Jupyter notebook - which in its simplest form is an electronic notebook consisting of a series of cells (like this one) which can contain a mix of text, code, output, metadata and potentially even interactive elements. Today we're working in Jupyter Lab - which is an environment which combines an interface to notebooks with a file explorer (left) and enables the integration of a variety of other tools.
##### Should you use notebooks?

Jupyter notebooks can be a good way to organise prototype workflows, and are often a good mechanism for sharing and explaining your code in a way which invites conversation and interaction (hence using them here!). Notably though, they're not necessarily the solution for everything. While you can construct workflows and models through Jupyter notebooks, they are more difficult to manage relative to standalone scripts and libraries when it comes to version management, integration and automation. For this reason it's suggested that once you have something working well, consider writing it up as a separate script or even a Python library/module!

##### Using Notebooks for Today (if you haven't seen them before)

The key thing to note for today is that it's common to find a mix of text cells like this one (typically written in [Markdown](https://www.markdownguide.org/) for easy markup of text) and code cells (scroll down a bit, they'll have a grey background). While it's not necessary for today, knowing a bit of markdown syntax can help structure notes and documentation accompanying your code. 

Code cells are not static - here on Binder you can run them (`Shift-Enter` or use the <i class="fas fa-play"></i> button), edit and re-run them! We encourage you to edit, change and break things within reason to get to know the tools (you can always restart Binder!).

You can tell which cells are being executed by the notation on the left of it - cells already run will have a number (e.g. `[1]`) noting the order in which it was run, cells yet to run will have an asterisk (`[*]`) and cells which haven't been executed will have empty brackets (`[ ]`). Also check the small circle in the upper right - if it's <i class="far fa-circle"></i> then it's stopped/hasn't started executing, if it's <i class="fas fa-circle"></i> it's trying to execute something/busy. If you get stuck and it looks like nothing's happening the kernel might have stalled; you can restart it under the `Kernel` menu to the top left, using `Restart Kernel...`.

<div class='alert alert-warning'> <font color="black"><b>Note:</b> Binder will not save your progress or changes! If you want to keep a modified notebook, you can right click and download from the file browser on the left (or, in Binder - you can also click the download link provided above).</font></div>



In [ ]:
"Run me with Shift-Enter/Cmd-Enter!"

In [ ]:
2 + 1

---
### What is Python?

[Python](https://www.python.org/) is a high-level multi-purpose programming language. It's freely and openly available and you'll be able to find a distribution which can run on just about any system (e.g. 'micropython' runs on bare-metal for tiny microcontrollers). There is a large community which uses Python, the majority of which revolves around open-source projects. You can use Python as a fancy calculator, build websites, run servers, build machine learning models, image black holes or provide testing and code generation for an [embedded software framework for NASA](https://github.com/nasa/fprime).

Python is an *interpreted* language, which means that rather than being compiled (like e.g. C, C++ and Fortran) it's read, interpreted and executed as needed. For this reason, it'll typically be a bit slower for most task (but not necessarily by much), but it also makes it much less complex to get into, read and run. When working with numerically intense workflows, you're often actually running code which was written in a more performant language in the background - and this bridges a large part of the gap between language 'performance'. Notably, however, Python tends to be written to be later read (or at least it can and should be) - and the accessibility together with it's flexibility are some of the key reasons it's so widely used.

You can run Python from the terminal, but typically we want to either write and execute programs (e.g. like 'scripts'; Python is often termed a 'scripting language') or play to the language's strengths and execute code interactively (e.g. in these notebooks!). To do this we need some kind of editor - whether it be notepad, Jupyter Notebooks or a dedicated development environment. While Python is often distributed with some kind of editor, many people have their own favourites - and it tends to depend a bit on what you're doing (e.g. I use 'VS Code', but write these workshops/demonstrations in Jupyter notebooks).

### Geochemical Data Science in Python - `Pandas` and `matplotlib`

You don't have to build everything yourself - there is a vast digital library of Python packages you can use for ready-made solutions:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Working with Geochemical Data - `pyrolite`

In [ ]:
import pyrolite.geochem

df = pd.read_csv('../data/basalts/Ueki2018.csv')
df.columns

#### Geochemical Data Visualisation - Spider Plots, Ternary Plots, +

In [ ]:
import pyrolite.plot

In [ ]:
df[["Pb208Pb204", "Nd143Nd144"]].pyroplot.scatter(c=df["Class"])

In [ ]:
fig, ax = plt.subplots(1)

for g, gdf in df.groupby("Class"):
    ax = gdf[["Fe2O3", "MgO", "TiO2"]].pyroplot.scatter(ax=ax, alpha=0.2, label=g)

ax.legend()


In [ ]:
ax = df.pyrochem.normalize_to("PM_PON", units="ppm").pyroplot.spider(
    unity_line=True, alpha=0.05, c=df["Class"]
)
ax.set(ylabel="X / Primitive Mantle (Palme and O'Neill)");


##### Plot Templates 

In [ ]:
from pyrolite.util.classification import TAS

df["Na2O + K2O"] = df["Na2O"] + df["K2O"]
cm = TAS()

fig, ax = plt.subplots(1)
cm.add_to_axes(ax, alpha=0.5, linewidth=0.5, zorder=-1, add_labels=True)
df[["SiO2", "Na2O + K2O"]].pyroplot.scatter(ax=ax, c="k", alpha=0.2, axlabels=False)
plt.show()

In [ ]:
df["TAS"] = cm.predict(df)
df["Rocknames"] = df.TAS.apply(lambda x: cm.fields.get(x, {"name": None})["name"])
df["Rocknames"].sample(10)  # randomly check 10 sample rocknames

In [ ]:
fig, ax = plt.subplots(1)

cm.add_to_axes(
    ax,
    alpha=0.5,
    linewidth=0.0,
    zorder=-2,
    add_labels=False,
    which_ids=np.unique(df["TAS"]),
    fill=True,
    facecolor=[0.9, 0.8, 1.0],
)
cm.add_to_axes(ax, alpha=0.5, linewidth=0.5, zorder=-1, add_labels=True)
df[["SiO2", "Na2O + K2O"]].pyroplot.scatter(
    ax=ax, c=df["TAS"], alpha=0.7, axlabels=False
)

In [ ]:
spinel_df = pd.read_csv("../data/spinel/Schoneveld2020.csv")
spinel_df = spinel_df.rename(
    columns={
        c: c.replace("_apfu", "").replace("Fe3", "Fe3+")
        for c in spinel_df.columns
        if "_apfu" in c
    }
)
spinel_df.columns

In [ ]:
from pyrolite.util.classification import SpinelTrivalentTernary

spinel_cm = SpinelTrivalentTernary()
spinel_df["ClassifiedPhase"] = spinel_cm.predict(spinel_df)

fig, ax = plt.subplots(1)
ax = spinel_cm.add_to_axes(
    ax,
    alpha=0.5,
    linewidth=0.0,
    zorder=-2,
    add_labels=False,
    which_ids=np.unique(spinel_df["ClassifiedPhase"]),
    fill=True,
    facecolor=[0.9, 0.8, 1.0],
)
ax = spinel_cm.add_to_axes(ax, alpha=0.5, linewidth=0.5, zorder=-1, add_labels=True)
ax = spinel_df[spinel_cm.axis_components].pyroplot.scatter(
    ax=ax, c=spinel_df["ClassifiedPhase"]
)

### Into Machine Learning for Mineral Chemistry

Some of the key things to watch out for:
* Handling below detection limit data, especially for trace elements 
* The majority of variation in mineral chemistry is controlled by crystal-chemical factors (in response to geological processes, but in a constrained way); e.g. the first principal components will pick this up.

#### Model Concepts in `scikit-learn`

One of the most commonly used frameworks for machine learning in Python is `scikit-learn`, which predominantly focuses on training machine learning models or pipelines from tabular data.  At it's core, you can think of `scikit-learn` as functions which estimate a quantity or label (in code, typically named `y`) based on another set of predictor variables (in code, typically named `X`), such that the model approximates the function $f$ in $y=f(X)$.

In `scikit-learn`, the mechanism for training and using model is typically along the lines of (and most commonly exactly as follows):

```python
model = ModelClassName(<initial_configuation_parameters>) # instantiate a model
model.fit(X, y)                                           # train the model to approximate the relationship between X and y
```

To subsequently use this model to make predictions, you'll typically use code along the lines of:

```python
predictions = model.predict(X_new) 
```

In [ ]:
df = pd.read_csv("../data/rutile/Plavsa2018.csv").set_index("Grain ID", drop=True)
df.pyrochem.elements = df.pyrochem.elements.apply(
    pd.to_numeric, errors="coerce"
).astype(float)
df.columns


In [ ]:
df["Phase ID"].value_counts() / df.index.size

In [ ]:
X = df.pyrochem.elements
y = df["Phase ID"]


Because we have some null-data in X, and most `scikit-learn` models don't handle this scenario, we'll need to filter them out (at least for now, and maybe come back to improve how we handle this later):

In [ ]:
fltr = ~(pd.isnull(X).sum(axis=1) > 0)
X, y = X.loc[fltr, :], y.loc[fltr]


We can see that we still retain about 95% of our data, which is good to know:

In [ ]:
fltr.sum() / fltr.size

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# setting random state so everyone gets the same results
random_state = 17
classifier = RandomForestClassifier(random_state=random_state)
classifier.fit(X, y)


Let's make some predictions on our dataset and see how well we do:

In [ ]:
predictions = classifier.predict(X)
predictions[:10]


In [ ]:
df.loc[fltr, ["Phase ID"]].assign(Predictions=predictions)

Our classifier does very well, maybe too well!

In [ ]:
classifier.score(X, y)

It looks pretty good, but this wasn't really a fair test - we're using the same data to train the model as we are to examine it; this is referred to as 'data leakage' in ML modeling. Instead what we want to do is keep back a hold-out-set for testing the model which we don't use for training the model.

`scikit-learn` has some built in tools for this, thankfully. These allow us to specify the proportion of data we keep for testing, and whether we 'stratify' the dataset such that we have roughly equal proportions of labels in our training and testing sets (in this case, that's probably a good idea). We can also chain `scikit-learn` components together to make a 'pipeline', adding additional preprocessing steps or bringing together/splitting parts of our dataset.

We don't do quite as well here, but this is closer to what it might look like in the real world (assuming the things we find are similar to those in our training set, of course..):

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from pyrolite.util.skl.transform import LogTransform


XX_train, XX_test, yy_train, yy_test = train_test_split(
    X,
    y,
    stratify=y,
    test_size=0.3,
)

transform = LogTransform()  # log-transform the data
clf = RandomForestClassifier(random_state=random_state)  # our classifer model
pipe = make_pipeline(transform, clf)
pipe


In [ ]:
pipe.fit(XX_train, yy_train).score(XX_test, yy_test)

Another way to look at how well a classifier model performs is on a per-class basis, such as that used in confusion matrix. In this instance is shows us some new information we wouldn't necessarily have seen otherwise - that our predictions for brookite are the worst, and it's mostly because it's getting misclassified as anatase (~4% of the time):

In [ ]:
from pyrolite.util.skl.vis import plot_confusion_matrix

fig, ax = plt.subplots(1, figsize=(5, 4))

plot_confusion_matrix(pipe, XX_test, yy_test, ax=ax, normalize=True)

ax.set_title("Polymorph Classifier\nConfusion Matrix")
ax.set(xlabel="Predicted Polymorph", ylabel="True Polymorph");


Given we're working with random forests, a handy thing to do at this stage might be to look at the relative feature importances (there are reasons why these might not be the most accurate picture, but it gives an idea of how the model is working):

In [ ]:
pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)

<div class='alert alert-success'> <b> Optional Exercise:</b><br> We've blitzed through an example using the mineral geochemistry to identify the titanium phases they represent, but we can also use the information in the dataset  to construct a model around whether those phases are related to mineralization.<br><br>Try altering the code above to make a model based on the 'Mineralized' column of the datset!</div>


----

| [**Overview**](./00_overview.ipynb) | [**EDA**](./01_EDA.ipynb) | [**Using `sklearn` Models**](./02_LoadModels.ipynb) | [**Making Predictions**](./03_Predictions.ipynb)|
| -- | -- | -- | -- |